# Introduction à PyTorch : Modèle, Fonction de coût et Optimiseur

---- 

Dans cette partie nous allons voir un peu plus en profondeur comment se servir des différents optimiseur et fonction de coût au sein des modèles. En effet cette partie est donc consacrée aux différentes pièces de notre puzzle.

On va donc voir les différentes étapes d'une pipeline en PyTorch !

Nous avons vu un exemple de modèle de régression linéaire plus tôt dans lequel nous avons définie notre fonction $forward()$ et $loss()$ et notre mise-à-jour des poids à la main. Cependant dans la réalité le framework au travers du modèle, définie automatiquement la fonction forward et des fonctions de loss sont déjà implémentées. De plus des optimiseurs aussi sont implémentés pour effectuer des algorithmes basés sur la descente de gradient avec des variantes particulières plus rapide.

Ainsi tous ces outils seront à utiliser pour se servir correctement de PyTorch (mise à part les fonctions de loss qui peuvent être données à la main dans certains cas très précis...)

----

## Définition de la pipeline :

En PyTorch une pipeline se définie telle que :

- 1/- On construit le modèle (inputs, output, layer -> définition de la fonction $forward()$)
- 2/- On définie/construit notre fonction de coût (loss) et notre optimiseur
- 3/- On construit notre boucle d'entraînement (comme pour la régression linéaire) :
    - Calcul du $forward()$
    - Rétropropagation avec les calculs de gradients
    - Mise-à-jour des poids

On peut rajouter des éléments graphiques pour suivre l'évolution de l'apprentissage par exemple aussi.

----

## Exemple avec une régression linéaire (cas du perceptron simple) :

Pour cette partie nous allons illustrer l'utilisation de chaque composantes de la pipeline en construisant un modèle avec un perceptron simple 

fonction d'agrégation linéaire telle que $$f : x → w_1x_1 + w_2x_2 + b$$ 

In [ ]:
import torch
import torch.nn as nn

### Préparation des données (toujours en premier) :

In [ ]:
X = torch.tensor([[1], [2], [3], [4], [5], [6], [7], [8]], dtype = torch.float32)
Y = torch.tensor([[2], [4], [6], [8], [10], [12], [14], [16]], dtype = torch.float32)

n_samples, n_features = X.shape # X.shape renvoi un tuple (8,1) c'est pourquoi au lieu de créer une liste, on créé 8 listes de 1 élément dans nos données (ça fait 8 lignes et 1 colonne))

print(n_samples, n_features) 

### Définition du modèle : 

In [ ]:
class LinearRegression(nn.Module):
    
    def __init__(self, input_size, output_size): # On précise les dimensions d'entrée et de sortie
        
        super(LinearRegression, self).__init__() # On appelle le constructeur de la classe parente qui est nn.Module
        
        self.lin = nn.Linear(input_size, output_size) # On crée une couche linéaire ou perceptron (ie la fonction d'agrégation est linéaire, c'est notre forward())
        
        # Remarque : Ici les poids sont initialisés aléatoirement et avec un argument require_grad=True, ce qui signifie qu'ils sont suivis par le gradient
    
    
    def forward(self, x): # On définit la fonction forward qui est appelée lors de l'inférence
        return self.lin(x) # On applique la couche linéaire à l'entrée x et c'est tout
    
    # C'est très important de comprendre que définir les layers ce n'est pas la même chose que les appliquer
    # Notre modèle possède un certain nombre de couches mais c'est l'application des couches qui détermine la structure que l'on souhaite pour notre réseau.

In [ ]:
input_size = n_features
output_size = n_features # En effet on donne un x et on nous recrache un y en dimension 1 donc on a bien une dimension de sortie de 1

model = LinearRegression(input_size, output_size)

print(f'Prediction before training: f(5) = {model(X[4]).item():.3f}') # On affiche la prédiction avant l'entraînement sur la donnée 5

# Remarque : on peut utiliser notre modèle pour prédire n'importe quelle donnée à condition qu'elle soit sous forme d'un tenseur :
print(model(torch.tensor([5], dtype = torch.float32))) # On a bien les même valeurs prédites
# On remarque que le résultat issue du modèle possède un élément grad_fn -> Logique car on a appliqué une couche linéaire à notre entrée donc une opération avec W et b 

### Définition de l'optimiseur et de la loss :

In [ ]:
# On initialise maintenant les hyperparamètres du modèle :

learning_rate = 0.01
n_epoch = 100 # On a vu que 20 epochs étaient largement suffisants pour converger

loss = nn.MSELoss() # On définit la fonction de coût qui est l'erreur quadratique moyenne
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate) # On utilise une descente de gradient stochastique et on rentre les paramètres avec model.parameters() en plus du lr

La définition de la loss et de l'optimiseur n'est pas plus compliqué que ça. Le plus difficile reste de savoir quelle loss est la plus intéressante selon le problème. En ce qui concerne l'optimiseur on va favoriser 90% du temps Adam qui possède de meilleures performances dans la totalité des cas sur des gros jeux de données. Ici pour une régression linéaire on va utiliser une simple descente de gradient stochastique.

### Définition de la boucle d'entraînement :

In [ ]:
for epoch in range(n_epoch):
    
    y_pred = model(X) # On calcule la prédiction du modèle, en clair c'est notre forward()
    
    l = loss(Y, y_pred) # On calcule la fonction de coût
    
    # On a effectué toutes opérations dont on a besoin pour l'optimisation, on va donc appliquer la descente de gradient :
    
    l.backward() # On calcule les gradients de la fonction de coût par rapport aux paramètres du modèle
    
    optimizer.step() # On met à jour les paramètres du modèle en fonction des gradients calculés précédemment (partie with torch.no_grad() inutile ici, c'est tout automatique)
    
    optimizer.zero_grad() # On remet à zéro les gradients pour la prochaine itération -> ça faut pas l'oublier par contre...
    
    if (epoch + 1) % 10 == 0:
        w,b = model.parameters() # On récupère les paramètres du modèle pour l'affichage
        print(f'epoch {epoch + 1}: w = {w[0][0].item():.3f}, loss = {l:.8f}') # On affiche les paramètres du modèle et la fonction de coût à chaque itération
        
print(f'Prediction after training: f(5) = {model(X[4]).item():.3f}') # On affiche la prédiction après l'entraînement sur la donnée 5


----

## Conclusion

Voici comment s'articule une pipeline (très similaire à ce qu'on a vu dans autograd.ipynb du coup)

La marge de progression sur l'utilisation de PyTorch réside alors dans la manière dont on construit les modèles. On favorise toujours l'utilisation de classes modèles héritant de $nn.Module$

Ainsi le prochain cours sera à propos de spécificités concernant la création de modèles et d'architectures avec notamment la création des layers et la façon dont on veut les appliquer dans la fonction $forward()$